## PARAMETERS

In [379]:
user_id = "64916754"                           # Define your account id         
age = "-".join(["18"])                         # Define the age range
country = "-".join(["BE"])                     # Define the countries
interests = "-".join(["6003031026255"])        # Define the interests

## IMPORT LIBRARIES

In [17]:
import base64
from string import ascii_letters, digits, punctuation, printable
from IPython.display import Image, display, HTML
from time import sleep
from collections import Counter
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from copy import copy
import requests
import json
import os
import platform
from datetime import datetime
import numpy as np
import pandas as pd
from ast import literal_eval
from requests import get

## START SCRAPING

In [358]:
# start the Chrome driver
driver = webdriver.Chrome()

In [143]:
# function to log in on your instagram account
def login_facebook(driver, username, password):
    
    driver.get("https://www.facebook.com/")
    
    login_cred = driver.find_elements_by_class_name("inputtext")
    
    login_cred[0].send_keys(username)
    login_cred[1].send_keys(password)
    
    login_button = driver.find_element(by=By.ID, value="loginbutton")
    
    login_button.click()

In [359]:
login_facebook(driver, "charlesverleyen@hotmail.com", "")

In [373]:
driver.get("https://www.facebook.com/ads/audience-insights/people?act="\
           + user_id + "&age=" + age + "-&country=" + country + "&interests=" + interests)

In [146]:
print(driver.title)

Audience Insights


In [147]:
percent_gender_data = driver.find_elements_by_class_name("_kdk")
percent_data_woman = int(percent_gender_data[0].text[:2])
percent_data_man = int(percent_gender_data[1].text[:2])

In [148]:
percent_gender_whole = driver.find_elements_by_class_name("_5n4z")
percent_whole_women = int(percent_gender_whole[0].text[:2])
percent_whole_men = int(percent_gender_whole[1].text[:2])

In [149]:
percent_whole_women

49

In [150]:
education = driver.find_elements_by_class_name("_5tt6")

In [151]:
education[0].text

'Single\nIn a Relationship\nEngaged\nMarried\n-16%+8%+0%+5%'

In [152]:
education[0].find_elements_by_tag_name("canvas")[0].get_attribute("width")

'366'

In [154]:
# get the base64 representation of the canvas image (the part substring(21) is for removing the padding "data:image/png;base64")
base64_image = driver.execute_script("return document.querySelectorAll('canvas')[0].\
toDataURL('image/png').substring(21);")
base64_image_1 = driver.execute_script("return document.querySelectorAll('canvas')[1].\
toDataURL('image/png').substring(21);")
base64_image_2 = driver.execute_script("return document.querySelectorAll('canvas')[2].\
toDataURL('image/png').substring(21);")

# decode the base64 image
output_image = base64.b64decode(base64_image)
output_image_1 = base64.b64decode(base64_image_1)
output_image_2 = base64.b64decode(base64_image_2)

# save to the output image
with open("image.png", 'wb') as f:
   f.write(output_image)
with open("image_1.png", 'wb') as f:
   f.write(output_image_1)
with open("image_2.png", 'wb') as f:
   f.write(output_image_2)

In [155]:
import io
import os

# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

GOOGLE_APPLICATION_CREDENTIALS = "/home/charly/Downloads/Training-afd38eccfc75.json"

client = vision.ImageAnnotatorClient()

## Demographics - Age and Gender chart

In [156]:
file_name = os.path.join(os.path.dirname('__file__'),'/home/charly/Desktop/Facebook Scraping/image.png')

In [157]:
with io.open(file_name, 'rb') as image_file:
    content = image_file.read()
image = types.Image(content=content)
response = client.text_detection(image=image)
texts = response.text_annotations

In [162]:
dict_text = {}
for i in texts[1:]:
    dict_text[i.description] = (i.bounding_poly.vertices[0].x, i.bounding_poly.vertices[0].y)

In [168]:
import operator
sorted_x = sorted(dict_text.items(), key=operator.itemgetter(1))

In [263]:
dict_age_dist = {}
temp_array = []
temp = ""
for i in sorted_x:
    if(re.search("[-+]", i[0])):
        dict_age_dist[i[0]] = {}
        temp = i[0]
    
    if(re.match('\d\d96',i[0])):
        if(i[1][1] > 100):
            dict_age_dist[temp]['male'] = re.sub(r'96','%',i[0], flags=re.IGNORECASE)
        else:
            dict_age_dist[temp]['female'] = re.sub(r'96','%',i[0], flags=re.IGNORECASE)
    
    if(re.search(r'%',i[0])):
        if(i[1][1] > 100):
            dict_age_dist[temp]['male'] = i[0]
        else:
            dict_age_dist[temp]['female'] = i[0]

In [262]:
dict_age_dist

{'18-24': {'male': '17%'},
 '25-34': {'female': '16%', 'male': '25%'},
 '35-44': {'female': '30%', 'male': '26%'},
 '45-54': {'female': '33%', 'male': '23%'},
 '55-64': {'female': '8%', 'male': '6%'},
 '65+': {'female': '5%', 'male': '3%'}}

## Demographics - Relationship Status chart

In [265]:
file_name2 = os.path.join(os.path.dirname('__file__'),'/home/charly/Desktop/Facebook Scraping/image_1.png')

In [303]:
with io.open(file_name2, 'rb') as image_file:
    content = image_file.read()
image = types.Image(content=content)
response = client.text_detection(image=image)
texts = response.text_annotations

In [304]:
dict_text = {}
for i in texts[1:]:
    dict_text[i.description] = (i.bounding_poly.vertices[0].x, i.bounding_poly.vertices[0].y)

In [305]:
for (i, k) in dict_text.items():
    dict_text[i] = k[0]
sorted_x = sorted(dict_text.items(), key=operator.itemgetter(1))
percentage = []
for i in sorted_x:
    if re.search(r'%',i[0]) is None:
        percentage.append(re.sub(r'96','%',i[0], flags=re.IGNORECASE))
    else:
        percentage.append(i[0])

In [306]:
education = driver.find_elements_by_class_name("_4dfp")

In [307]:
rel = []
for i in education[:4]:
    rel.append(i.text)

In [310]:
dict_relationship = {}
for i,j in zip(rel, percentage):
    dict_relationship[i] = j

In [311]:
dict_relationship

{'Engaged': '4%',
 'In a Relationship': '34%',
 'Married': '39%',
 'Single': '23%'}

## Demographics - Education Level chart

In [312]:
file_name3 = os.path.join(os.path.dirname('__file__'),'/home/charly/Desktop/Facebook Scraping/image_2.png')

In [326]:
with io.open(file_name3, 'rb') as image_file:
    content = image_file.read()
image = types.Image(content=content)
response = client.text_detection(image=image)
texts = response.text_annotations

In [327]:
dict_text = {}
for i in texts[1:]:
    dict_text[i.description] = (i.bounding_poly.vertices[0].x, i.bounding_poly.vertices[0].y)

In [329]:
for (i, k) in dict_text.items():
    dict_text[i] = k[0]
sorted_x = sorted(dict_text.items(), key=operator.itemgetter(1))
percentage = []
for i in sorted_x:
    if re.search(r'%',i[0]) is None:
        percentage.append(re.sub(r'96','%',i[0], flags=re.IGNORECASE))
    else:
        percentage.append(i[0])

In [330]:
ed = []
for i in education[4:]:
    ed.append(i.text)

In [332]:
dict_relationship = {}
for i,j in zip(ed, percentage):
    dict_relationship[i] = j

In [333]:
dict_relationship

{'High School': '30%', 'University': '63%', 'University (postgraduate)': '7%'}

## Scrape Job Title and Industries

In [349]:
see_all_button = driver.find_elements_by_class_name("_5xp8")
see_all_button[0].click()

In [350]:
jobs = driver.find_elements_by_class_name("_54_6")
jobs = jobs[0].find_elements_by_tag_name("tr")

In [351]:
jobs_array = []
for i in jobs[1:]:
    jobs_list = i.find_elements_by_tag_name("td")
    temp_array = []
    for j in jobs_list:
        temp_array.append(j.text)
    jobs_array.append(temp_array)

In [352]:
jobs_array

[['Installation and repair services', '19%', '+90%'],
 ['Transport and moving', '14%', '+75%'],
 ['Farming, fishing and forestry', '10%', '+67%'],
 ['Architecture and engineering', '14%', '+56%'],
 ['Construction and extraction', '3%', '+50%'],
 ['Production', '17%', '+42%'],
 ['Life, physical and social sciences', '10%', '+25%'],
 ['Healthcare and medical services', '13%', '+8%'],
 ['Arts, entertainment, sport and media', '16%', '+7%'],
 ['Cleaning and maintenance services', '3%', '+0%'],
 ['Business and finance', '10%', '+0%'],
 ['Protective services', '1%', '+0%'],
 ['Administrative services', '17%', '-11%'],
 ['Sales', '17%', '-11%'],
 ['IT and technical services', '7%', '-12%'],
 ['Computation and mathematics', '7%', '-12%'],
 ['Management', '14%', '-12%'],
 ['Personal care and at-home services', '7%', '-12%'],
 ['Food and restaurants', '4%', '-33%'],
 ['Education and libraries', '9%', '-36%'],
 ['Community and social services', '7%', '-42%'],
 ['Legal services', '1%', '-50%']]

## Scrape Page Likes

In [380]:
driver.get("https://www.facebook.com/ads/audience-insights/interests?act="\
           + user_id + "&age=" + age + "-&country=" + country + "&interests=" + interests)

In [381]:
driver.get("https://www.facebook.com/ads/audience-insights/interests?act=64916754&age=18-&country=BE")

In [422]:
see_all_buttons = driver.find_elements_by_class_name("_5xp8")

In [423]:
see_all_buttons[0].click()
see_all_buttons[1].click()

In [404]:
interests_likes = driver.find_elements_by_class_name("_54_6")

In [406]:
interests_lists = interests_likes[0].find_elements_by_tag_name("tr")

In [407]:
interests_array = []
for i in interests_lists:
    jobs_list = i.find_elements_by_tag_name("td")
    temp_array = []
    for j in jobs_list:
        temp_array.append(j.text)
    interests_array.append(temp_array)

In [409]:
likes_lists = interests_likes[1].find_elements_by_tag_name("tr")

In [416]:
likes_array = []
for i in likes_lists[1:]:
    jobs_list = i.find_elements_by_tag_name("td")
    temp_array = []
    for j in jobs_list:
        temp_array.append(j.text)
    likes_array.append(temp_array)

In [448]:
interests_array.insert(0,["Number", "Category", "Page name" ])

In [449]:
interests_array

[['Number', 'Category', 'Page name'],
 ['1', 'Recreation & sport website', 'We Are Tennis'],
 ['2', 'Jewellery/Watches', 'Ice-Watch'],
 ['3', 'Electronics company', 'Samsung'],
 ['4', 'Food & drink', 'Coca-Cola'],
 ['5', 'Food delivery service', 'Takeaway.com'],
 ['6', 'Supermarket/Convenience store', 'Ferrero Rocher'],
 ['7', 'Film/Television studio', '20th Century FoxWarner Bros. Pictures'],
 ['8', 'Clothing (brand)', '3SuissesH&MHunkemöller'],
 ['9', 'Shopping centre', 'IKEA'],
 ['10', 'Film', 'AvatarStep Up MovieFast & Furious']]

In [444]:
likes_array.insert(0,["Page", "Relevance", "Audience","Facebook", "Affinity"])

In [445]:
likes_array

[['Page', 'Relevance', 'Audience', 'Facebook', 'Affinity'],
 ['We Are Tennis', '1', '139.3K', '134.9K', '282x'],
 ['Fun Radio', '2', '137.1K', '135.1K', '277x'],
 ['3Suisses', '3', '171.4K', '168.9K', '277x'],
 ['Ice-Watch', '4', '176.3K', '174.1K', '277x'],
 ['Samsung', '5', '321.5K', '319.2K', '275x'],
 ['Kinder', '6', '158.9K', '157.9K', '275x'],
 ['Thuis', '7', '180.9K', '179.8K', '275x'],
 ['Coca-Cola', '8', '374.6K', '372.6K', '274x'],
 ['Takeaway.com', '9', '163.7K', '162.9K', '274x'],
 ["Ben & Jerry's", '10', '161.4K', '160.6K', '274x']]

In [ ]:
jobs_array = []
for i in jobs[1:]:
    jobs_list = i.find_elements_by_tag_name("td")
    temp_array = []
    for j in jobs_list:
        temp_array.append(j.text)
    jobs_array.append(temp_array)

## Scrape Locations

In [ ]:
driver.get("https://www.facebook.com/ads/audience-insights/geography?act="\
           + user_id + "&age=" + age + "-&country=" + country + "&interests=" + interests)

In [429]:
driver.get("https://www.facebook.com/ads/audience-insights/geography?act=64916754&age=18-&country=BE")

In [430]:
see_all_button = driver.find_elements_by_class_name("_5xp8")

In [431]:
see_all_button[0].click()

In [432]:
locations = driver.find_elements_by_class_name("_54_6")

In [434]:
locations_lists = locations[0].find_elements_by_tag_name("tr")

In [439]:
locations_array = []
for i in locations_lists[1:]:
    jobs_list = i.find_elements_by_tag_name("td")
    temp_array = []
    for j in jobs_list:
        temp_array.append(j.text)
    locations_array.append(temp_array)

In [442]:
locations_array.insert(0,["Towns/Cities", "Selected Audience", "Compare"])

In [443]:
locations_array

[['Towns/Cities', 'Selected Audience', 'Compare'],
 ['Halle, Flemish Region, Belgium', '0.2%', '+100%'],
 ['Zandvliet, Flemish Region, Belgium', '0.2%', '+100%'],
 ['Rocourt, Wallonia, Belgium', '0.3%', '+50%'],
 ['Ostend, Flemish Region, Belgium', '0.5%', '+25%'],
 ['Molenbeek-Saint-Jean, Brussels, Belgium', '0.6%', '+20%'],
 ['Genk, Flemish Region, Belgium', '0.8%', '+14%'],
 ['Hasselt, Flemish Region, Belgium', '1%', '+10%'],
 ['Leuven, Flemish Region, Belgium', '2%', '+6%'],
 ['Liège, Wallonia, Belgium', '2%', '+6%'],
 ['Namur, Wallonia, Belgium', '2%', '+6%'],
 ['Lüttich, Wallonia, Belgium', '3%', '+4%'],
 ['Brussels, Brussels, Belgium', '7%', '+3%'],
 ['Antwerp, Flemish Region, Belgium', '4%', '+3%'],
 ['Gent, Flemish Region, Belgium', '6%', '+2%'],
 ['Aarschot, Flemish Region, Belgium', '0.2%', '+0%'],
 ['Alost, Flemish Region, Belgium', '0.3%', '+0%'],
 ['Andenne, Wallonia, Belgium', '0.2%', '+0%'],
 ['Arlon, Wallonia, Belgium', '0.4%', '+0%'],
 ['Ath, Wallonia, Belgium', '0.4%

In [ ]:
driver.close()